## Installation stuff

In [ ]:
# Install Mantisshrimp package
!pip install -q git+git://github.com/lgvaz/mantisshrimp.git

In [ ]:
# Install cocoapi and albumentations packages
!pip install -q 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -q albumentations --upgrade

# Install fastai and/or Pytorch-Lightning
!pip install -q fastai2
!pip install -q pytorch-lightning

In [ ]:
!pip install -q wandb

In [ ]:
! wandb login

In [ ]:
!nvidia-smi

## Load the dataset

In [ ]:
from mantisshrimp.imports import *
from mantisshrimp import *
import albumentations as A

In [ ]:
import wandb

In [ ]:
path = datasets.pets.load()

In [ ]:
parser = datasets.pets.parser(path)

In [ ]:
CLASSES = datasets.pets.CLASSES

In [ ]:
data_splitter = RandomSplitter([.8, .2])

In [ ]:
train_records, valid_records = parser.parse(data_splitter)

In [ ]:
# ImageNet stats
imagenet_mean, imagenet_std = IMAGENET_STATS

train_tfms = AlbuTransform(
    [
        A.LongestMaxSize(384),
        A.RandomSizedBBoxSafeCrop(320, 320, p=0.3),
        A.HorizontalFlip(),
        A.ShiftScaleRotate(rotate_limit=20),
        A.RGBShift(always_apply=True),
        A.RandomBrightnessContrast(),
        A.Blur(blur_limit=(1, 3)),
        A.Normalize(mean=imagenet_mean, std=imagenet_std),
    ]
)

In [ ]:
valid_tfms = AlbuTransform(
    [
        A.LongestMaxSize(384),
        A.Normalize(mean=imagenet_mean, std=imagenet_std),
    ]
)

In [ ]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
model = MantisFasterRCNN(num_classes= len(CLASSES))

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
train_dl = model.dataloader(train_ds, batch_size=16, num_workers=4, shuffle=True)
valid_dl = model.dataloader(valid_ds, batch_size=16, num_workers=4, shuffle=False)

## Ligthning Code

In [ ]:
# import lightning engine provided by the mantisshrimp modules
from mantisshrimp.engines.lightning import *
from pytorch_lightning.loggers import WandbLogger

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'parameters': {
        'learning_rate': {
            'values': [0.002, 0.001]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        }
    },
    'program' : 'train'
}

In [ ]:
wandb_logger = WandbLogger()
wandb.init(project="mantis_demo1")

W&B Run: https://app.wandb.ai/oke-aditya/mantis_demo1/runs/13caseo8

In [ ]:
class LightModel(RCNNLightningAdapter):
    def configure_optimizers(self):
        opt = SGD(self.parameters(), 2e-4, momentum=0.9)
        return opt

In [ ]:
trainer = Trainer(max_epochs=3, gpus=1, logger=wandb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
light_model = LightModel(model)

In [ ]:
def train():
    trainer.fit(light_model, train_dl, valid_dl)

In [ ]:
if __name__ == 'main':
    train()

In [ ]:
sweep_id = wandb.sweep(sweep_config,project='mantis_demo1')

Create sweep with ID: wm11k1wa
Sweep URL: https://app.wandb.ai/oke-aditya/mantis_demo1/sweeps/wm11k1wa


In [ ]:
wandb.agent(sweep_id, train)
# !wandb agent grz4dwda